In [5]:
import pandas as pd
from backtesting import Backtest, Strategy
from ta import add_all_ta_features

In [6]:
data = pd.read_csv('data.csv')
data['date'] = pd.to_datetime(data['date'])
data

,date,Open,Low,High,Close,Volume
0,2024-05-06 09:30:00,89.3900,89.0600,90.6800,90.5400,43561890
1,2024-05-06 09:45:00,90.5700,90.3200,91.6000,91.5800,37915780
2,2024-05-06 10:00:00,91.6000,90.9400,91.6900,91.1200,24796720
3,2024-05-06 10:15:00,91.1000,90.8300,91.3900,91.3100,19628230
4,2024-05-06 10:30:00,91.3100,91.1500,91.6300,91.4200,16087600
...,...,...,...,...,...,...
6658,2025-05-05 12:30:00,113.7650,113.6282,114.0550,114.0213,2792553
6659,2025-05-05 12:45:00,114.0101,113.8550,114.0600,114.0193,2456978
6660,2025-05-05 13:00:00,114.0100,113.8800,114.1000,113.9400,2924088
6661,2025-05-05 13:15:00,113.9400,113.8800,114.1600,114.1350,82135144


In [7]:
data_ta = add_all_ta_features(data, open='Open', high='High', low='Low', close='Close', volume='Volume')
data_ta

,date,Open,Low,High,Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2024-05-06 09:30:00,89.3900,89.0600,90.6800,90.5400,43561890,3.603267e+07,43561890,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2024-05-06 09:45:00,90.5700,90.3200,91.6000,91.5800,37915780,7.276359e+07,81477670,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.148664,1.142117,1.148664
2,2024-05-06 10:00:00,91.6000,90.9400,91.6900,91.1200,24796720,5.986929e+07,56680950,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.502293,-0.503559,0.640601
3,2024-05-06 10:15:00,91.1000,90.8300,91.3900,91.3100,19628230,7.388946e+07,76309180,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.208516,0.208299,0.850453
4,2024-05-06 10:30:00,91.3100,91.1500,91.6300,91.4200,16087600,7.590041e+07,92396780,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.120469,0.120396,0.971946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6658,2025-05-05 12:30:00,113.7650,113.6282,114.0550,114.0213,2792553,3.036901e+09,2367541412,0.000053,6.710196e+04,...,-0.062150,-0.051311,-0.010840,-24.339679,-29.789587,5.449908,113.691111,0.207672,0.207457,25.934725
6659,2025-05-05 12:45:00,114.0101,113.8550,114.0600,114.0193,2456978,3.038383e+09,2365084434,0.007262,5.681398e+04,...,-0.042023,-0.049453,0.007430,-22.887153,-28.409100,5.521947,113.736954,-0.001754,-0.001754,25.932516
6660,2025-05-05 13:00:00,114.0100,113.8800,114.1000,113.9400,2924088,3.037053e+09,2362160346,0.028682,1.557195e+04,...,-0.031334,-0.045829,0.014495,-20.082287,-26.743738,6.661451,113.755479,-0.069550,-0.069574,25.844930
6661,2025-05-05 13:15:00,113.9400,113.8800,114.1600,114.1350,82135144,3.104522e+09,2444295490,0.522352,2.301398e+06,...,-0.008943,-0.038452,0.029509,64.096038,-8.575783,72.671820,113.788662,0.171143,0.170996,26.060305


In [8]:
training_data = data_ta.iloc[0:int(len(data) / 2), :]
testing_data = data_ta.iloc[int(len(data) / 2):, :]

In [9]:
class Strat(Strategy):
    current_position_direction = None
    
    def init(self):
        ...
        
    def next(self):    
        if all([
            self.data['trend_adx'][-1] > 25,
            self.data['trend_adx_pos'][-1] > self.data['trend_adx_neg'][-1],
        ]):
            if self.current_position_direction != 'long':
                self.position.close()
                self.buy()
                self.current_position_direction = 'long'
                
        if all([
            self.data['Close'][-1] < self.data['trend_psar_down_indicator'][-1],
            self.data['Close'][-2] >= self.data['trend_psar_down_indicator'][-2],
            self.data['trend_vortex_ind_pos'][-1] < self.data['trend_vortex_ind_neg'][-1],
            self.data['trend_vortex_ind_pos'][-2] >= self.data['trend_vortex_ind_neg'][-2],
        ]):
            # Close long position on confirmed reversal
            if self.current_position_direction == 'long':
                self.position.close()
                self.current_position_direction = None
                
        if all([
            self.data['momentum_pvo'][-1] > 0,
            self.data['trend_ema_fast'][-1] < self.data['trend_ema_slow'][-1],
            self.data['trend_ema_fast'][-2] >= self.data['trend_ema_slow'][-2],
        ]):
            # If not already short, close current position and open short
            if self.current_position_direction != 'short':
                self.position.close()
                self.sell()
                self.current_position_direction = 'short'
                
        if all([
            self.data['trend_cci'][-1] < 0,
            self.data['momentum_stoch_rsi'][-1] < 0.5
        ]):
            if self.current_position_direction == 'short':
                self.position.close()
                self.current_position_direction = None

In [10]:
bt = Backtest(training_data, Strat, cash=1_000_000, commission=.002, exclusive_orders=True, hedging=False)
bt.run()

Start                                     0.0
End                                    3330.0
Duration                               3330.0
Exposure Time [%]                   99.159412
Equity Final [$]               2277597.379156
Equity Peak [$]                2283278.929156
Return [%]                         127.759738
Buy & Hold Return [%]                47.55909
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.734358
Avg. Drawdown [%]                   -2.129978
Max. Drawdown Duration                  860.0
Avg. Drawdown Duration              44.150685
# Trades                                 48.0
Win Rate [%]                        47.916667
Best Trade [%]                      35.098042
Worst Trade [%]                    -10.973256
Avg. Trade [%]                    

In [11]:
bt = Backtest(testing_data, Strat, cash=1_000_000, commission=.002, exclusive_orders=True, hedging=False)
bt.run()

Start                                  3331.0
End                                    6662.0
Duration                               3331.0
Exposure Time [%]                   98.679472
Equity Final [$]                2294321.01022
Equity Peak [$]                  2648146.4689
Return [%]                         129.432101
Buy & Hold Return [%]              -14.266652
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.613726
Avg. Drawdown [%]                   -2.024367
Max. Drawdown Duration                  489.0
Avg. Drawdown Duration               33.14433
# Trades                                 51.0
Win Rate [%]                        56.862745
Best Trade [%]                      17.318308
Worst Trade [%]                     -7.602868
Avg. Trade [%]                    

In [12]:
bt = Backtest(data, Strat, cash=1_000_000, commission=.002, exclusive_orders=True, hedging=False)
bt.run()

Start                                     0.0
End                                    6662.0
Duration                               6662.0
Exposure Time [%]                   99.114513
Equity Final [$]               5119155.428056
Equity Peak [$]                5908619.843376
Return [%]                         411.915543
Buy & Hold Return [%]               26.242213
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.734358
Avg. Drawdown [%]                   -2.108643
Max. Drawdown Duration                  860.0
Avg. Drawdown Duration              38.628743
# Trades                                 99.0
Win Rate [%]                        52.525253
Best Trade [%]                      35.098042
Worst Trade [%]                    -10.973256
Avg. Trade [%]                    